Домашняя работа №3 по курсу "Продвинутое машинное обучение"

выполнил Денисов Артем, MADE-ML-22

In [1]:
from random import shuffle, seed, choices, random

import numpy as np

## 0. Подготовка данных и вспомогательные функции

In [53]:
ALPHABET = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя' + ' '

In [54]:
def open_text_file(filepath):
    """
    открытие файла с текстом и преобразование его в 1 строку
    """
    text = []
    with open(filepath, encoding='utf-8') as file:
        for row in file:
            text.append(row)
    return ''.join(text)

In [55]:
def preprocess_text(text, alphabet):
    """
    удаление из текста цифр и знаков препинания
    """
    new_text = []
    for letter in text.lower():
        if letter in alphabet:
            new_text.append(letter)
    return ''.join(new_text)

In [120]:
def letter_accuracy(true_text, decoded_text, alphabet):
    """
    функция для проверки качества модели, считает долю отгаданны букв. 
    Работает при условии, что известен правильный вариант расшифровки
    """
    all_answers = 0
    correct_answers = 0
    for letter_true, letter_decoded in zip(true_text, decoded_text):
        if letter_decoded in alphabet:
            if letter_true == letter_decoded:
                correct_answers += 1
            all_answers += 1
    return correct_answers / all_answers

In [57]:
corpus = open_text_file('WarAndPeace.txt')
processed_corpus = preprocess_text(corpus, ALPHABET)
train_frequency = build_letter_frequency(processed_corpus, ALPHABET)
len(corpus), len(processed_corpus)

(717873, 649795)

In [119]:
"""
тестовый кусок текста из предисловия к Анне Карениной, на котором будут проверяться модели
"""
test_text = "«Мне теперь так ясна моя мысль, – говорил Толстой в 1877 году, завершая работу над романом. – Так, в „Анне Карениной“ я люблю мысль семейную, в „Войне и мире“ любил мысль народную вследствие войны 1812 года…».[25] Конечно, и в «Войне и мире» есть «семейная хроника», и в «Анне Карениной» есть «картины народной жизни». Но для каждого романа нужна была своя, особенная любовь. «Мысль семейная» для 70-х годов XIX века особенно актуальна. «Одна из самых важных задач в этом текущем, для меня, например, – признавался Ф.М. Достоевский, – молодое поколение, и вместе с тем современная русская семья, которая, я предчувствую это, далеко не такова, как всего еще двадцать лет назад».[26]"
test_text = preprocess_text(test_text, ALPHABET)
test_text

'мне теперь так ясна моя мысль  говорил толстой в  году завершая работу над романом  так в анне карениной я люблю мысль семейную в войне и мире любил мысль народную вследствие войны  года конечно и в войне и мире есть семейная хроника и в анне карениной есть картины народной жизни но для каждого романа нужна была своя особенная любовь мысль семейная для х годов  века особенно актуальна одна из самых важных задач в этом текущем для меня например  признавался фм достоевский  молодое поколение и вместе с тем современная русская семья которая я предчувствую это далеко не такова как всего еще двадцать лет назад'

In [59]:
def shuffle_text(text, alphabet):
    """
    функция для зашифровки текста
    """
    seed(42)
    shuffled_text = []
    
    shuffled_alphabet_list = [letter for letter in alphabet]
    alphabet_list = [letter for letter in alphabet]
    shuffle(shuffled_alphabet_list)
    shuffle_mapping = {a: b for a, b in zip(alphabet_list, shuffled_alphabet_list)} 
    for letter in text:
        if letter in alphabet:
            shuffled_text.append(shuffle_mapping[letter])
        else:
            shuffled_text.append(letter)
    return ''.join(shuffled_text), shuffle_mapping

In [60]:
shuffled_text, shuffle_mapping = shuffle_text(test_text, ALPHABET)
shuffled_text

'изыжяыёыхэжяенжбазежиьбжидашэжжльуьхкшжяьшаяьщжужжльчмжпеуыхфебжхетьямжзечжхьиезьижжяенжужеззыжнехызкзьщжбжшртшржидашэжаыиыщзмржужуьщзыжкжикхыжшрткшжидашэжзехьчзмржуашычаяукыжуьщзджжльчежньзыъзьжкжужуьщзыжкжикхыжыаяэжаыиыщзебжвхьзкнежкжужеззыжнехызкзьщжыаяэжнехякзджзехьчзьщжйкпзкжзьжчшбжнейчьльжхьиезежзмйзежтдшежауьбжьаьтыззебжшртьуэжидашэжаыиыщзебжчшбжвжльчьужжуынежьаьтыззьженямешэзежьчзежкпжаеидвжуейздвжпечеъжуж яьижяынмгыижчшбжиызбжзеёхкиыхжжёхкпзеуешабжсижчьаяьыуанкщжжиьшьчьыжёьньшызкыжкжуиыаяыжажяыижаьухыиыззебжхмаанебжаыиэбжньяьхебжбжёхычъмуаяумрж яьжчешыньжзыжяеньуежненжуаыльжыгыжчуечюеяэжшыяжзепеч'

## 1. Реализуйте базовый частотный метод по Шерлоку Холмсу
○	подсчитайте частоты букв по корпусам (пунктуацию и капитализацию можно просто опустить, а вот пробелы лучше оставить);
○	возьмите какие-нибудь тестовые тексты (нужно взять по меньшей мере 2-3 предложения, иначе вряд ли сработает), зашифруйте их посредством случайной перестановки символов;
○	расшифруйте их таким частотным методом.

In [122]:
def build_letter_frequency(text, alphabet):
    """
    создание частотного словаря букв на большом тексте
    """
    frequency_dict = {letter: 0 for letter in alphabet}
    for letter in text:
        frequency_dict[letter] += 1
    return frequency_dict

In [61]:
def make_decoding_mapping(train_frequency, test_frequency):
    """
    функция для сопоставления двух алфавитов. Каждый из алфавитом сортируется по частоте, 
    предполагется, если порядок букв в отсортированном алфавите совпадает, то и буквы алфавитов совпадают
    """
    train_frequency_list = [(value, key) for key, value in train_frequency.items()]
    train_frequency_list.sort(reverse=True)
    test_frequency_list = [(value, key) for key, value in test_frequency.items()]
    test_frequency_list.sort(reverse=True)
    decoding_mapping = dict()
    for i in range(len(test_frequency_list)):
        train_counter, train_letter = train_frequency_list[i]
        test_counter, test_letter = test_frequency_list[i]
        decoding_mapping[test_letter] = train_letter
    return decoding_mapping

In [62]:
def decode_text(text, mapping):
    """
    функция для расшифровки текста при заданном словаре между зашифрованным алфавитом и правильным алфавитом
    """
    decoded_text = []
    for letter in text:
        decoded_text.append(mapping[letter])
    return ''.join(decoded_text)

In [63]:
test_frequency = build_letter_frequency(shuffled_text, ALPHABET)
decode_mapping = make_decoding_mapping(train_frequency, test_frequency)

In [64]:
decoded_text = decode_text(shuffled_text, decode_mapping)
print(decoded_text)

сие лейерп лам униа соу сьнвп  чоторкв ловнлоя т  чодг батерщау раыолг иад росаиос  лам т аиие мареикиоя у взывз сьнвп несеяигз т тояие к скре взыкв сьнвп иародигз тнведнлтке тояиь  чода моиехио к т тояие к скре енлп несеяиау шроикма к т аиие мареикиоя енлп марлкиь иародиоя жкбик ио дву маждочо росаиа игжиа ыьва нтоу оноыеииау взыотп сьнвп несеяиау дву ш чодот  тема оноыеиио амлгавпиа одиа кб насьш тажиьш бадах т цлос лемгюес дву сеиу иайрксер  йркбиатавну фс донлоетнмкя  соводое йомовеике к тсенле н лес нотресеииау ргннмау неспу молорау у йредхгтнлтгз цло давемо ие ламота мам тнечо еюе дтадэалп вел иабад


In [66]:
print(f"{letter_accuracy(test_text, decoded_text, ALPHABET):.3f}")

0.489


Расшифровка текста еще нечитаемая, хотя почти половину букв получилось угадать

## 2.	Биграммы
Вряд ли в результате получилась такая уж хорошая расшифровка, разве что если вы брали в качестве тестовых данных целые рассказы. Но и Шерлок Холмс был не так уж прост: после буквы E, которая действительно выделяется частотой, дальше он анализировал уже конкретные слова и пытался угадать, какими они могли бы быть. Я не знаю, как запрограммировать такой интуитивный анализ, так что давайте просто сделаем следующий логический шаг:

In [68]:
def build_letter_frequency_bigram(text, n=2):
    """
    функция для построения частотного словаря биграмм
    """
    frequency_dict = dict()
    for letter_index in range(len(text) - n + 1):
        bigram = text[letter_index:letter_index + n]
        if bigram in frequency_dict.keys():
            frequency_dict[bigram] += 1
        else:
            frequency_dict[bigram] = 1
    return frequency_dict

In [69]:
def decode_text_bigram(text, mapping, n=2):
    """
    Не придумал красивого способа для биграмм, похожего на базовую модель. 
    Предположил, что из словаря биграмм можно определить букву, которая на 0 месте в биграмме
    """
    decoded_text = []
    for letter_index in range(len(text) - n + 1):
        bigram = text[letter_index:letter_index + n]
        decoded_text.append(mapping[bigram][0])
    decoded_text.append(mapping[bigram][1])
    return ''.join(decoded_text)

In [70]:
train_frequency_bigram = build_letter_frequency_bigram(processed_corpus)
test_frequency_bigram = build_letter_frequency_bigram(shuffled_text)
len(train_frequency_bigram), len(test_frequency_bigram)

(827, 199)

In [71]:
decode_mapping = make_decoding_mapping(train_frequency_bigram, test_frequency_bigram)
decoded_text_bigram = decode_text_bigram(shuffled_text, decode_mapping)

In [72]:
decoded_text_bigram

'яаеное тл нолдбуби  нио оиле накретыввноаюко  аннакяуси дхткорориорелс и шровт гвоннолдане оаеп лйсн г  ботевдез оиле ту ьолаазанае лаев  ичйетевгвв оиле  илоядааза лрсткаисеае леанакя  п нахэг в анае лаев  ичйеоокк ту ьолироавонну  в ане оаеп лйсн г  оокк п лик еа илоядг  ли нн  г маноп мипск ровт и  аз и шдрт тлеиолсмеосоиротевррс  оиле ту ьолироманоадакяпрннахт  лсмеосог ел лымепи ляди в итщуоадавм еади   еаанмновоноттус омано ьсуо игтырьтинжты нидвмюловоомпскочб вяе н начяпчежии арснсев асьокоетано отмрий ьсоирорсыив роту сеоп еотиробожтйсдшк ка азмно м мрт   аенол рв п лда уык омсембв о жк тркс ио  ч'

In [73]:
alphabet = ''.join(set(decoded_text_bigram))
print(f"{letter_accuracy(test_text, decoded_text_bigram, alphabet):.3f}")

0.074


Правильно определить частоты биграмм на маленьком куске текста, и я предполагаю, есть более красивый способ расшифровки, и результат получился хуже, чем в базовой модели

## 3. MCMM алгоритм	
Но и это ещё не всё: биграммы скорее всего тоже далеко не всегда работают. Основная часть задания — в том, как можно их улучшить:

○	предложите метод обучения перестановки символов в этом задании, основанный на MCMC-сэмплировании, но по-прежнему работающий на основе статистики биграмм;

○	реализуйте и протестируйте его, убедитесь, что результаты улучшились


0. Биграммы - это марковская цепь, вторая буква, которая следует за первой и появляется с вероятностью, которую можно оценить
1. На корпусе текстов я оцениваю по биграммам, какова вероятность появления буквы i+1 после буквы i. Для этого я считаю количество таких сочетаний, а затем делю это на примерно длину текста, чтобы отнормировать
2. Для оценки правдоподобия текста используется функция, где просто перемножаются вероятности каждой из биграмм. Чтобы не оперировать в Python очень маленькими числами на грани ошибки округления, вероятности логарифмируются и складываются
3. Расчет начинается со словаря со случайным сопоставлением алфавита зашифрованного сообщения и алфавита, обученного на большом тексте.
4. Текст расшифровывается при текущем сопоставлении алфавитов, расчитывается его правдоподобие
5. Далее выбираются два случайных симввола и переставляются в словаре
6. Текст расшифровывается на попытке перестановки, рассчитывается его правдоподобие. (Первоначально я принимал перестановку, только если правдоподобие росло, но в ходе обучения редко происходили     перестановки и качество получалось очень плохим)
7. После этого изменил модель и стал принимать перестановку с вероятностью p(current) / p(best). Перестановки стали случаться чаще и в результаты некоторых обучений модели стали лучше, чем в базовом варианте
8. Если перестановка принимается, происходит обновления словаря между двумя алфавитами. Далее происходит переход к пункту 5 и так 15 000 итераций
8. Прогоны чаще всего сходились в локальном оптимуме и давали плохое качество модели, чтобы обойти, происходит 30 попыток обучения и в финале принимается результат с максимальным правдоподобием.
10. Остановился на количесте итераций для перестановок 15000 - это примерно 600 перестановок и 30 попыток обучения, чтобы получить хотя бы 1 результат близкий к оптимуму

In [116]:
def estimate_probabilites(corpus, alphabet, ngram=2):
    """
    функция построения частотного словаря биграмм и н-грамм
    """
    n_text = len(corpus)
    prob_dict = dict()
    for letter_index in range(n_text - ngram):
        n_letter = corpus[letter_index:letter_index + ngram]
        if n_letter in prob_dict.keys():
            prob_dict[n_letter] += 1
        else:
            prob_dict[n_letter] = 1

    
    for key in prob_dict.keys():
        prob_dict[key] /= (n_text - n_gram)
    return prob_dict

In [94]:
def count_probability(text, prob_dict, ngram=2):
    """
    функция для оценки правдоподобия текста при заданном частотном словаре биграмм
    """
    n_text = len(text)
    log_prob = 0
    for letter_index in range(n_text - ngram + 1):
        n_letters = text[letter_index:letter_index + ngram]
        if n_letters in prob_dict.keys():
            log_prob += np.log(prob_dict[n_letters])
        else:
            log_prob += -10000
    return log_prob

In [95]:
prob_dict = estimate_probabilites(processed_corpus, ALPHABET)
len(prob_dict)

827

In [99]:
def decode_text_mcmm(text, prob_dict, alphabet, n_steps=1000, ngram=2):
    """
    функция, в которой происходит 1 попытка обучения с количеством итераций n_steps. 
    Соответствует шагам 3-8 из описания
    """
    text_alphabet = list(set(text))
    if len(text_alphabet) < len(alphabet):
        for i in range(len(alphabet) - len(text_alphabet)):
            text_alphabet.append("pad" + str(i))
    current_log_proba = count_probability(text, prob_dict, ngram=ngram)
    mapping_for_decoding = {text_letter: letter for text_letter, letter in zip(text_alphabet, alphabet)}
    accepted_steps = 0
    for i in range(n_steps):
        letter_one, letter_two = choices(text_alphabet, k=2)
        test_mapping = mapping_for_decoding.copy()
        test_mapping[letter_one], test_mapping[letter_two] = test_mapping[letter_two], test_mapping[letter_one]
        try_decoded_text = decode_text(text, test_mapping)
        new_prob = count_probability(try_decoded_text, prob_dict, ngram=ngram)
        
        prob_condition = np.exp(new_prob - current_log_proba)
        if prob_condition > random():
            accepted_steps += 1
            mapping_for_decoding = test_mapping
            current_log_proba = new_prob
    return mapping_for_decoding, current_log_proba, accepted_steps / n_steps

In [123]:
"""
код для запуска новых циклов обучения и выбора лучшего из них, соответствует пункту 9 описания
"""
best_proba = -100000
for i in range(30):
    mapping, proba, accepted = decode_text_mcmm(shuffled_text, prob_dict, ALPHABET, n_steps=10000)
    if proba > best_proba:
        best_proba = proba
        best_mapping = mapping
        print(f"log_proba: {proba:0.2f}, accept ratio: {accepted:0.2f}")
decoded_text = decode_text(shuffled_text, best_mapping)    
print(decoded_text)
accuracy = letter_accuracy(test_text, decoded_text, ALPHABET)
print(f"точность расшифровки:{accuracy:.3f}")

<ipython-input-99-57df6514c700>:17: RuntimeWarning: overflow encountered in exp
  prob_condition = np.exp(new_prob - current_log_proba)


log_proba: -3875.84, accept ratio: 0.04
log_proba: -3852.71, accept ratio: 0.04
log_proba: -3822.23, accept ratio: 0.04
log_proba: -3425.65, accept ratio: 0.04
log_proba: -3395.59, accept ratio: 0.04
мне теперь так ясна моя мысль  говорил толстой в  гобу заверцая радоту наб романом  так в анне карениной я людлю мысль семейную в войне и мире людил мысль наробную вслебствие войны  гоба конечно и в войне и мире есть семейная хроника и в анне карениной есть картины наробной жизни но бля кажбого романа нужна дыла своя осоденная людовь мысль семейная бля х гобов  века осоденно актуальна обна из самых важных забач в этом текушем бля меня например  признавался фм бостоевский  молобое поколение и вместе с тем современная русская семья которая я пребчувствую это балеко не такова как всего еше бвабщать лет назаб
0.949


In [109]:
"""
Текст из домашней работы, для проверки
"""
test = "დჳჵჂႨშႼႨშჂხჂჲდႨსႹႭჾႣჵისႼჰႨჂჵჂႨႲႹႧჲჂႨსႹႭჾႣჵისႼჰႨჲდႩჳჲႨჇႨႠჲႹქႹႨჳႹႹჱჶდსჂႽႨႩႹჲႹႭႼჰႨჵდქႩႹႨႲႭႹႧჂჲႣჲიႨჳႩႹႭდდႨშჳდქႹႨშႼႨშჳდႨჳხდჵႣჵჂႨႲႭႣშჂჵისႹႨჂႨႲႹჵჇႧჂჲდႨჾႣႩჳჂჾႣჵისႼჰႨჱႣჵჵႨეႣႨႲႹჳჵდხსდდႨႧდჲშდႭჲႹდႨეႣხႣსჂდႨႩჇႭჳႣႨႾႹჲႽႨႩႹსდႧსႹႨႽႨსჂႧდქႹႨსდႨႹჱდჶႣნ"

28

In [127]:
best_proba = -10000
for i in range(30):
    mapping, proba, accepted = decode_text_mcmm(test, prob_dict, ALPHABET, n_steps=10000)
    if proba > best_proba:
        best_proba = proba
        print(f"log_proba: {proba:0.2f}, accept ratio: {accepted:0.2f}")
        best_mapping = mapping
decoded_text = decode_text(test, best_mapping)    
print(decoded_text)

<ipython-input-99-57df6514c700>:17: RuntimeWarning: overflow encountered in exp
  prob_condition = np.exp(new_prob - current_log_proba)


log_proba: -1315.34, accept ratio: 0.08
log_proba: -1286.20, accept ratio: 0.07
log_proba: -1257.11, accept ratio: 0.07
если вы вимите норзальный или подти норзальный текст у этого соочшения который легко продитать скорее всего вы все смелали правильно и полудите заксизальный чалл ба послемнее детвертое бамание курса хотя конедно я нидего не очешаю


Со второй попытки получилась достаточно хорошая расшифровка